In [54]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import copy
import folium
import pyeasyga.pyeasyga as pyeasyga
from pyeasyga.pyeasyga import GeneticAlgorithm

In [2]:
#CARGA DE DATOS
#Se reutiliza el código de carga de datos del proyecto anterior

def load_distance_time_dic(path):
    data = pd.read_csv(path)
    distance = {}
    time = {}
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen, destino] = float(data.iloc[i, 2])
        time[origen, destino] = float(data.iloc[i, 3])
    return distance, time
5
def load_vehicles(path):
    data = pd.read_csv(path)
    vehicles = {}
    for i in range(len(data)):
        id = int(data.iloc[i, 0])
        capacity = int(data.iloc[i, 1])
        ran = float(data.iloc[i, 2])
        vehicles[id] = (capacity, ran)
    return vehicles

def load_demand(path):
    data = pd.read_csv(path)
    demand_dic = {}
    for i in range(len(data)):
        id = int(data.iloc[i, 1])
        demand = float(data.iloc[i, 2])
        demand_dic[id] = demand
    return demand_dic

def load_capacity(depositPath):
    data= pd.read_csv(depositPath)
    N = len(data)
    capacity_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
    
    return capacity_dic

def load_coordinates(depotsPath, clientsPath):
    coord = {}
    depot = pd.read_csv(depotsPath)
    client = pd.read_csv(clientsPath)
    for i in range(len(depot)):
        id = int(depot.iloc[i, 1])
        lat = float(depot.iloc[i, 3])
        long = float(depot.iloc[i, 2])
        coord[id] = [lat, long]
    for j in range(len(client)):
        id = int(client.iloc[j, 1])
        lat = float(client.iloc[j, 4])
        long = float(client.iloc[j, 3])
        coord[id] = [lat, long]
    return coord


In [3]:
distancia,time_dic = load_distance_time_dic('../Datos/caso3.csv')
vehiculos = load_vehicles('../Datos/vehicles.csv')
demanda = load_demand('..\Datos\clients.csv')
coord = load_coordinates('..\Datos\depots.csv', '..\Datos\clients.csv')
print(f"Distancia: {distancia}")
print(f"Vehiculos: {vehiculos}")
print(f"Demanda: {demanda}")
print(f"Coordenadas: {coord}")
print(f"Time: {time_dic}")



Distancia: {(1, 2): 33061.2, (1, 3): 10160.7, (1, 4): 6452.9, (1, 5): 21415.6, (1, 6): 16924.1, (1, 7): 20126.9, (1, 8): 11361.2, (1, 9): 4329.9, (1, 10): 31289.5, (1, 11): 13659.0, (1, 12): 26223.2, (1, 13): 16323.5, (1, 14): 10339.3, (1, 15): 20151.1, (1, 16): 9069.0, (1, 17): 13001.1, (1, 18): 11585.9, (1, 19): 5611.1, (1, 20): 35182.4, (1, 21): 5382.8, (1, 22): 15080.8, (1, 23): 15604.8, (1, 24): 17375.6, (1, 25): 18066.4, (1, 26): 15519.7, (1, 27): 22099.1, (1, 28): 21116.6, (1, 29): 18265.0, (1, 30): 24161.7, (1, 31): 22148.0, (1, 32): 11296.0, (1, 33): 7930.6, (1, 34): 25061.6, (1, 35): 23030.4, (1, 36): 18832.4, (1, 37): 30974.4, (1, 38): 15263.2, (1, 39): 26853.9, (1, 40): 19256.8, (1, 41): 7132.5, (1, 42): 18337.4, (1, 43): 14053.1, (1, 44): 14476.5, (1, 45): 22832.7, (1, 46): 22156.7, (1, 47): 20648.7, (1, 48): 28891.8, (1, 49): 23837.9, (1, 50): 14339.2, (1, 51): 18236.1, (1, 52): 11772.5, (1, 53): 11658.8, (1, 54): 23255.9, (1, 55): 17080.0, (1, 56): 11591.1, (1, 57): 2327

In [7]:
# Parámetros
depot_id = 1  # depósito con demanda 0
pf = 15000
ft = 5000
cm = 700
gv = 0.411458

def evaluar_individuo(rutas, demanda, vehiculos, distancia, pf, ft, cm, gv):
    claves_vehiculos = list(vehiculos.keys())
    total_cost = 0
    for v_id, ruta in enumerate(rutas):
        if len(ruta) <= 2:
            continue
        cap_max, rango_max = vehiculos[claves_vehiculos[v_id]]
        carga = 0
        distancia_total = 0

        for i in range(len(ruta) - 1):
            a, b = ruta[i], ruta[i+1]
            distancia_total += distancia.get((a, b), 9999999)
            if b != depot_id:
                carga += demanda.get(b, 0)

        penalizacion = 0
        if carga > cap_max:
            penalizacion += 1e6 * (carga - cap_max)
        if distancia_total > rango_max:
            penalizacion += 1e6 * (distancia_total - rango_max)

        costo_vehiculo = gv*pf * distancia_total + ft + cm + penalizacion
        total_cost += costo_vehiculo
    return total_cost

def generar_poblacion_inicial(n_individuos, clientes, vehiculos):
    poblacion = []
    n_vehiculos = len(vehiculos)
    for _ in range(n_individuos):
        rutas = [[] for _ in vehiculos]
        clientes_copy = clientes[:]
        random.shuffle(clientes_copy)
        # Asignar cada cliente exactamente a un vehículo
        for idx, c in enumerate(clientes_copy):
            v_id = idx % n_vehiculos  # asignación round robin para mejor distribución
            rutas[v_id].append(c)
        # Agregar depósito al inicio y fin de cada ruta
        rutas = [[depot_id] + r + [depot_id] if r else [] for r in rutas]
        poblacion.append(rutas)
    return poblacion

def mutar(rutas):
    nuevas = copy.deepcopy(rutas)
    # Elegir dos vehículos distintos
    v1, v2 = random.sample(range(len(rutas)), 2)
    if len(nuevas[v1]) > 2:
        cliente = random.choice(nuevas[v1][1:-1])
        nuevas[v1].remove(cliente)
        # Insertar cliente en una posición aleatoria en v2, excepto el depósito final
        pos = random.randint(1, len(nuevas[v2]) - 1) if len(nuevas[v2]) > 0 else 1
        nuevas[v2].insert(pos, cliente)
    return nuevas

def cruzar(p1, p2):
    hijo = copy.deepcopy(p1)
    v = random.randint(0, len(p1) - 1)
    if len(p1[v]) > 2 and len(p2[v]) > 2:
        sub1 = p1[v][1:-1]
        sub2 = p2[v][1:-1]
        split = len(sub1) // 2
        nuevo = [depot_id] + sub1[:split] + sub2[split:] + [depot_id]
        # Para evitar duplicados/remover clientes fuera de esta ruta,
        # hacemos limpieza después del cruce
        hijo[v] = nuevo

        # Extraer clientes asignados en la nueva ruta
        asignados = set(nuevo[1:-1])

        # Eliminar estos clientes de otras rutas para evitar duplicados
        for i in range(len(hijo)):
            if i != v:
                hijo[i] = [depot_id] + [c for c in hijo[i][1:-1] if c not in asignados] + [depot_id]

        # Insertar los clientes que quedaron fuera en la ruta v de forma aleatoria
        todos_clientes = set(c for ruta in p1 for c in ruta[1:-1])
        fuera = todos_clientes - asignados
        for c in fuera:
            pos = random.randint(1, len(hijo[v]) - 1)
            hijo[v].insert(pos, c)
    return hijo

def nodos_no_asignados(rutas, clientes, depot_id):
    asignados = set()
    for ruta in rutas:
        asignados.update(n for n in ruta if n != depot_id)
    no_asignados = set(clientes) - asignados
    return no_asignados

def run_ga(distance, demand, vehicles, coord, n_generaciones=200, n_poblacion=50, elitismo=5, prob_mutacion=0.2):
    demanda[depot_id] = 0
    clientes = [c for c in demand.keys() if c != depot_id]
    poblacion = generar_poblacion_inicial(n_poblacion, clientes, vehicles)
    best_indiv = []
    for gen in range(n_generaciones):
        puntuaciones = [(evaluar_individuo(ind, demand, vehicles, distance, pf, ft, cm, gv), ind) for ind in poblacion]
        puntuaciones.sort(key=lambda x: x[0])
        best_indiv.append(puntuaciones[0][0])
        elite = [ind for (_, ind) in puntuaciones[:elitismo]]

        nueva_poblacion = elite[:]
        while len(nueva_poblacion) < n_poblacion:
            p1, p2 = random.choices(elite, k=2)
            hijo = cruzar(p1, p2)
            if random.random() < prob_mutacion:
                hijo = mutar(hijo)
            # Validar que no haya nodos sin asignar, si hay los añadimos manualmente
            faltantes = nodos_no_asignados(hijo, clientes, depot_id)
            if faltantes:
                # Añadir faltantes a rutas aleatorias
                for f in faltantes:
                    v_id = random.randint(0, len(vehicles) -1)
                    # Insertar antes del depósito final
                    hijo[v_id].insert(-1, f)
            nueva_poblacion.append(hijo)

        poblacion = nueva_poblacion

    mejor_coste, mejor_sol = min([(evaluar_individuo(ind, demand, vehicles, distance, pf, ft, cm, gv), ind) for ind in poblacion], key=lambda x: x[0])

    faltantes_final = nodos_no_asignados(mejor_sol, clientes, depot_id)
    if faltantes_final:
        print("¡Atención! Algunos nodos no fueron asignados en la mejor solución:", faltantes_final)
    else:
        print("Todos los nodos fueron asignados correctamente.")

    print("Mejor costo:", mejor_coste)
    return mejor_sol, best_indiv



In [ ]:
depot_id = 1  # depósito con demanda 0
pf = 15000
ft = 5000
cm = 700
gv = 0.411458

class VRPGeneticAlgorithm:
    def __init__(self, distance, demand, vehicles, coord):
        self.distance = distance
        self.demand = demand
        self.vehicles = vehicles
        self.coord = coord
        self.clientes = [c for c in demand.keys() if c != depot_id]
        self.demand[depot_id] = 0
        
    def create_individual(self, data):
        """Crear un individuo (conjunto de rutas)"""
        n_vehiculos = len(self.vehicles)
        rutas = [[] for _ in range(n_vehiculos)]
        clientes_copy = self.clientes[:]
        random.shuffle(clientes_copy)
        
        # Asignar cada cliente exactamente a un vehículo (round robin)
        for idx, cliente in enumerate(clientes_copy):
            v_id = idx % n_vehiculos
            rutas[v_id].append(cliente)
        
        # Agregar depósito al inicio y fin de cada ruta
        rutas = [[depot_id] + ruta + [depot_id] if ruta else [] for ruta in rutas]
        return rutas

    def mutate_individual(self, individual, data=None):
        """Mutación: mover un cliente de un vehículo a otro"""
        nuevas_rutas = copy.deepcopy(individual)
        
        # Encontrar vehículos con rutas no vacías
        vehiculos_con_rutas = [i for i, ruta in enumerate(nuevas_rutas) if len(ruta) > 2]
        
        if len(vehiculos_con_rutas) < 2:
            return nuevas_rutas
        
        # Elegir dos vehículos distintos
        v1, v2 = random.sample(vehiculos_con_rutas, 2)
        
        if len(nuevas_rutas[v1]) > 2:
            # Seleccionar cliente aleatorio (excluyendo depósitos)
            cliente = random.choice(nuevas_rutas[v1][1:-1])
            nuevas_rutas[v1].remove(cliente)
            
            # Insertar cliente en posición aleatoria en v2
            if len(nuevas_rutas[v2]) == 0:
                nuevas_rutas[v2] = [depot_id, cliente, depot_id]
            else:
                pos = random.randint(1, len(nuevas_rutas[v2]) - 1)
                nuevas_rutas[v2].insert(pos, cliente)
        
        return nuevas_rutas

    def crossover_individuals(self, parent1, parent2, data=None):
        """Cruzamiento: intercambiar segmentos de rutas entre padres"""
        hijo1 = copy.deepcopy(parent1)
        hijo2 = copy.deepcopy(parent2)
        
        # Seleccionar vehículo aleatorio para cruzar
        v = random.randint(0, len(parent1) - 1)
        
        if len(parent1[v]) > 2 and len(parent2[v]) > 2:
            # Extraer rutas sin depósitos
            ruta1 = parent1[v][1:-1]
            ruta2 = parent2[v][1:-1]
            
            if ruta1 and ruta2:
                # Punto de cruce
                split1 = random.randint(0, len(ruta1))
                split2 = random.randint(0, len(ruta2))
                
                # Crear nuevas rutas cruzadas
                nueva_ruta1 = ruta1[:split1] + ruta2[split2:]
                nueva_ruta2 = ruta2[:split2] + ruta1[split1:]
                
                # Reconstruir rutas con depósitos
                hijo1[v] = [depot_id] + nueva_ruta1 + [depot_id] if nueva_ruta1 else []
                hijo2[v] = [depot_id] + nueva_ruta2 + [depot_id] if nueva_ruta2 else []
                
                # Limpiar duplicados y reasignar clientes faltantes
                hijo1 = self._reparar_solucion(hijo1)
                hijo2 = self._reparar_solucion(hijo2)
        
        return hijo1, hijo2

    def _reparar_solucion(self, rutas):
        """Reparar solución eliminando duplicados y asignando clientes faltantes"""
        # Encontrar todos los clientes asignados
        asignados = set()
        for ruta in rutas:
            for cliente in ruta:
                if cliente != depot_id:
                    asignados.add(cliente)
        
        # Eliminar duplicados
        rutas_limpias = []
        clientes_vistos = set()
        
        for ruta in rutas:
            nueva_ruta = [depot_id]
            for cliente in ruta[1:-1]:  # Excluir depósitos
                if cliente not in clientes_vistos:
                    nueva_ruta.append(cliente)
                    clientes_vistos.add(cliente)
            if len(nueva_ruta) > 1:
                nueva_ruta.append(depot_id)
                rutas_limpias.append(nueva_ruta)
            else:
                rutas_limpias.append([])
        
        # Asignar clientes faltantes
        faltantes = set(self.clientes) - clientes_vistos
        for cliente in faltantes:
            v_id = random.randint(0, len(rutas_limpias) - 1)
            if len(rutas_limpias[v_id]) == 0:
                rutas_limpias[v_id] = [depot_id, cliente, depot_id]
            else:
                rutas_limpias[v_id].insert(-1, cliente)
        
        return rutas_limpias

    def fitness_individual(self, individual, data):
        """Evaluar fitness de un individuo (menor es mejor)"""
        claves_vehiculos = list(self.vehicles.keys())
        total_cost = 0
        
        for v_id, ruta in enumerate(individual):
            if len(ruta) <= 2:  # Ruta vacía
                continue
                
            cap_max, rango_max = self.vehicles[claves_vehiculos[v_id]]
            carga = 0
            distancia_total = 0
            
            # Calcular distancia y carga de la ruta
            for i in range(len(ruta) - 1):
                a, b = ruta[i], ruta[i + 1]
                distancia_total += self.distance.get((a, b), 9999999)
                if b != depot_id:
                    carga += self.demand.get(b, 0)
            
            # Calcular penalizaciones por violación de restricciones
            penalizacion = 0
            if carga > cap_max:
                penalizacion += 1e6 * (carga - cap_max)
            if distancia_total > rango_max:
                penalizacion += 1e6 * (distancia_total - rango_max)
            
            # Costo total del vehículo
            costo_vehiculo = gv * pf * distancia_total + ft + cm + penalizacion
            total_cost += costo_vehiculo
        
        # pyeasyga maximiza fitness, por lo que devolvemos el negativo del costo
        return -total_cost

    def tournament_selection(self, population, tournament_size=3):
        """Selección por torneo"""
        def tournament_select():
            # Seleccionar individuos aleatorios para el torneo
            tournament = random.sample(population, min(tournament_size, len(population)))
            # Devolver el mejor individuo del torneo
            return max(tournament, key=lambda x: x.fitness)
        
        return tournament_select

    def _verificar_asignacion_completa(self, rutas):
        """Verificar que todos los clientes estén asignados"""
        asignados = set()
        for ruta in rutas:
            for cliente in ruta:
                if cliente != depot_id:
                    asignados.add(cliente)
        
        faltantes = set(self.clientes) - asignados
        return faltantes


def run_ga(distance, demand, vehicles, coord, 
                            n_generaciones=200, n_poblacion=50, 
                            prob_mutacion=0.2, prob_cruzamiento=0.8,
                            elitismo=5, tournament_size=3):
    """
    Versión mejorada del algoritmo genético con pyeasyga
    """
    # Crear instancia del problema VRP con verificaciones extendidas
    vrp = VRPGeneticAlgorithm(distance, demand, vehicles, coord)
    
    # Configurar algoritmo genético
    ga = GeneticAlgorithm(
        seed_data=None,
        population_size=n_poblacion,
        generations=n_generaciones,
        crossover_probability=prob_cruzamiento,
        mutation_probability=prob_mutacion,
        elitism=True,
        maximise_fitness=True
    )
    
    # Asignar funciones personalizadas
    ga.create_individual = vrp.create_individual
    ga.mutate_function = vrp.mutate_individual
    ga.crossover_function = vrp.crossover_individuals
    ga.fitness_function = vrp.fitness_individual
    
    fitness_history = []

    # Sobrescribir el método rank_population para guardar el mejor fitness
    original_rank_population = ga.rank_population
    def rank_and_record():
        original_rank_population()
        # El mejor fitness está en la primera posición después de ordenar
        fitness_history.append(ga.current_generation[0].fitness)
    ga.rank_population = rank_and_record
    
    # Ejecutar algoritmo genético
    print("Iniciando algoritmo genético mejorado...")
    ga.run()
    
    # Obtener mejor solución
    mejor_solucion = ga.best_individual()[1]
    mejor_fitness = -ga.best_individual()[0]
    
    # Verificaciones finales
    faltantes = vrp._verificar_asignacion_completa(mejor_solucion)
    if faltantes:
        print(f"¡Atención! Algunos nodos no fueron asignados: {faltantes}")
    else:
        print("Todos los nodos fueron asignados correctamente.")
    
    print(f"Mejor costo: {mejor_fitness}")
    
    # Obtener historial de fitness
    fitness_history = [-fitness for fitness in fitness_history]
    
    return mejor_solucion, fitness_history

In [58]:
import folium

def visualizar_rutas_folium(rutas, coord):
    # Crear mapa centrado en depósito o promedio de coordenadas
    lat_dep, lon_dep = coord[depot_id]
    m = folium.Map(location=[lat_dep, lon_dep], zoom_start=11, tiles='Cartodb Positron')

    colors = ['blue', 'green', 'cyan', 'magenta','olive', 'blue', 'orange', 'purple','red']
    icons = ['blue', 'green', 'lightblue', 'pink','lightgreen', 'blue', 'orange', 'darkpurple','red']

    for v, ruta in enumerate(rutas):
        if not ruta or len(ruta) < 2:
            continue

        # Construir lista de coordenadas para PolyLine: [(lat, lon), (lat, lon), ...]
        coords_ruta = [coord[n] for n in ruta]

        # Dibujar la ruta
        folium.PolyLine(
            coords_ruta,
            color=colors[v % len(colors)],
            weight=5,
            opacity=0.7,
            tooltip=f'Vehículo {v}'
        ).add_to(m)

        # Marcador inicio (depósito)
        folium.Marker(
            coords_ruta[0],
            popup="Inicio (Depósito)",
            icon=folium.Icon(color='black', icon='home')
        ).add_to(m)

        # Marcador fin de ruta
        folium.Marker(
            coords_ruta[-1],
            popup=f"Llegada Vehículo {v}",
            icon=folium.Icon(color=icons[v % len(icons)], icon='flag')
        ).add_to(m)

    return m


In [59]:
solucion, evol = run_ga(distancia, demanda, vehiculos, coord, n_generaciones=300)
def graficar_evolucion_costos(mejores_costos):
    plt.figure(figsize=(10, 5))
    plt.plot(mejores_costos, marker='o', color='green')
    plt.xlabel('Generación')
    plt.ylabel('Mejor costo')
    plt.title('Evolución del costo del mejor individuo')
    plt.grid(True)
    plt.show()

graficar_evolucion_costos(evol)
mapa = visualizar_rutas_folium(solucion, coord)
mapa

NameError: name 'run_ga' is not defined

## Analisis de Escalabilidad

## Comparación son soluciones previas (Pyomo)